# Core Objective

- `recommend`( user ) --> list of movies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

%run 'movies.ipynb'
%run 'tags.ipynb'
%run 'ratings.ipynb'
%run 'utils.ipynb'

pd.__version__

'0.20.1'

In [2]:
Tags = getTags()
allMovies = getMovies()
Ratings = getRatings()

verbose ==> False


In [3]:
Ratings.head(5)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


- Some basic Information retrieval on our data sets.

In [4]:
print('We have %d movies in our current movie set.' %(allMovies['movieId'].count()))

num_of_users = len(Ratings['userId'].unique());
num_of_movies = len(Ratings['movieId'].unique());
num_of_ratings = Ratings.shape[0]

print('We have %d ratings given by %d users for %d movies' % (num_of_ratings, num_of_users, num_of_movies))

filteredRating = Ratings[ Ratings['movieId'].isin(allMovies.movieId)]
print('Only %d ratings belong %d movies'%( filteredRating.shape[0], allMovies['movieId'].count()))

num_of_unique_users = len(filteredRating['userId'].unique())
print('Only %d unique user contributed in %d ratings.' % ( num_of_unique_users, filteredRating.shape[0]))

num_of_users_to_pick = 10;   # NUMBER OF USERS YOU WANT TO PICK FOR ANALYSIS.
users_picked = np.take(filteredRating['userId'].unique(), np.arange(1, num_of_users_to_pick + 1, 1))
rating_belong_to_picked_users = filteredRating[ filteredRating['userId'].isin(users_picked) ]
print('Lets pick %d user to play with.' % ( num_of_users_to_pick))
print('These %d users provided %d ratings'% (num_of_users_to_pick, rating_belong_to_picked_users.shape[0]))


We have 549 movies in our current movie set.
We have 100004 ratings given by 671 users for 9066 movies
Only 19175 ratings belong 549 movies
Only 663 unique user contributed in 19175 ratings.
Lets pick 10 user to play with.
These 10 users provided 224 ratings


In [12]:
_Ratings = rating_belong_to_picked_users.copy(deep=True)
_Ratings.shape

for userId, group in _Ratings.groupby(by=['userId']):
    print('(userId, total_rating) -> (%d, %d)' % (userId, group.shape[0]))

(userId, total_rating) -> (2, 74)
(userId, total_rating) -> (3, 14)
(userId, total_rating) -> (4, 33)
(userId, total_rating) -> (5, 18)
(userId, total_rating) -> (6, 5)
(userId, total_rating) -> (7, 40)
(userId, total_rating) -> (8, 17)
(userId, total_rating) -> (9, 12)
(userId, total_rating) -> (10, 6)
(userId, total_rating) -> (11, 5)


### Construct a `(user X movie)` matrix - Utility matrix

In [95]:
def __get_all_movie_titles(Movies):
    _titles = [];
    for index, movie_row in Movies.iterrows():
        _titles.append(movie_row.title)
    return _titles

def __get_empty_util_matrix(Movies):
    cols = ['userId'] + __get_all_movie_titles(Movies); 
    util_matrix = pd.DataFrame( columns = cols ); # create empty utility matrix.
    return util_matrix

In [103]:
def __get_default_ratings(num_of_cols, default_val = np.nan):
    empty_ratings = np.empty(num_of_cols);
    empty_ratings[:] = default_val;
    return empty_ratings

def __get_util_matrix_row(userId, _Ratings, Movies):
    user_ratings = [];
    util_matrix = __get_empty_util_matrix(Movies)
    ratings_by_user = _Ratings[_Ratings['userId'] == userId]
    
    num_of_cols = util_matrix.shape[1] -1;
    num_of_rows = util_matrix.shape[0]
    util_matrix.loc[num_of_rows + 1] = np.concatenate([[userId] , __get_default_ratings(num_of_cols)])

    
    # 2. Find ratings given by user, and fill it in map.
    for index, row in ratings_by_user.iterrows():
        # print(row)
        movie_user_has_rated = Movies[Movies['movieId'] == row.movieId]
        movie_name_user_has_rated = movie_user_has_rated['title'].iloc[0];
        util_matrix.loc[num_of_rows + 1][movie_name_user_has_rated] = row['norm_rating']
    
    return util_matrix

In [101]:
def __normalize_ratings(_Ratings):
    max_rating = _Ratings['rating'].max()
    min_rating = _Ratings['rating'].min() - 1; # since min value is inclusive.
    avg_rating = ((max_rating - min_rating) / 2)
    
    return _Ratings['rating'] - avg_rating

In [102]:
def prepare_util_matrix():
    util_matrix = __get_empty_util_matrix(allMovies)
    
    _Ratings['norm_rating'] = __normalize_ratings(_Ratings)
    
    userIds = _Ratings['userId'].unique()
    for userId in userIds:
        new_row__for_util_matrix = __get_util_matrix_row(userId, _Ratings, allMovies)
        util_matrix = util_matrix.append(new_row__for_util_matrix, ignore_index=True)
    return util_matrix

util_matrix = prepare_util_matrix()
util_matrix.head(10)

,userId,Toy Story (1995),Jumanji (1995),Grumpier Old Men (1995),Waiting to Exhale (1995),Father of the Bride Part II (1995),Heat (1995),Sabrina (1995),Tom and Huck (1995),Sudden Death (1995),GoldenEye (1995),"American President, The (1995)",Dracula: Dead and Loving It (1995),Balto (1995),Nixon (1995),Cutthroat Island (1995),Casino (1995),Sense and Sensibility (1995),Four Rooms (1995),Ace Ventura: When Nature Calls (1995),Money Train (1995),Get Shorty (1995),Copycat (1995),Assassins (1995),Powder (1995),Leaving Las Vegas (1995),Othello (1995),Now and Then (1995),Persuasion (1995),"City of Lost Children, The Cité des enfants perdus, La (1995)",Shanghai Triad Yao a yao yao dao waipo qiao (1995),Dangerous Minds (1995),Twelve Monkeys <a.k.a. 12 Monkeys> (1995),Babe (1995),Carrington (1995),Dead Man Walking (1995),Across the Sea of Time (1995),It Takes Two (1995),Clueless (1995),"Cry, the Beloved Country (1995)",Richard III (1995),Dead Presidents (1995),Restoration (1995),Mortal Kombat (1995),To Die For (1995),How to Make an American Quilt (1995),Seven <a.k.a. Se7en> (1995),Pocahontas (1995),When Night Is Falling (1995),"Usual Suspects, The (1995)",Mighty Aphrodite (1995),Lamerica (1994),"Big Green, The (1995)",Georgia (1995),Home for the Holidays (1995),"Postman, The <Postino, Il> (1994)","Confessional, The <Confessionnal, Le> (1995)","Indian in the Cupboard, The (1995)",Eye for an Eye (1996),Mr. Holland's Opus (1995),Don't Be a Menace to South Central While Drinking Your Juice in the Hood (1996),Two if by Sea (1996),Bio-Dome (1996),Lawnmower Man 2: Beyond Cyberspace (1996),French Twist <Gazon maudit> (1995),Friday (1995),From Dusk Till Dawn (1996),Fair Game (1995),Kicking and Screaming (1995),"Misérables, Les (1995)",Bed of Roses (1996),Screamers (1995),Nico Icon (1995),"Crossing Guard, The (1995)","Juror, The (1996)","White Balloon, The <Badkonake sefid> (1995)",Things to Do in Denver When You're Dead (1995),Antonia's Line <Antonia> (1995),Once Upon a Time... When We Were Colored (1995),Last Summer in the Hamptons (1995),Angels and Insects (1995),White Squall (1996),Dunston Checks In (1996),Black Sheep (1996),Nick of Time (1995),Mary Reilly (1996),Vampire in Brooklyn (1995),Beautiful Girls (1996),Broken Arrow (1996),In the Bleak Midwinter (1995),"Hate <Haine, La> (1995)",Shopping (1994),Heidi Fleiss: Hollywood Madam (1995),City Hall (1996),Bottle Rocket (1996),Mr. Wrong (1996),Unforgettable (1996),Happy Gilmore (1996),"Bridges of Madison County, The (1995)",Muppet Treasure Island (1996),Catwalk (1996),Braveheart (1995),Taxi Driver (1976),Rumble in the Bronx <Hont faan kui> (1995),Before and After (1996),Margaret's Museum (1995),Anne Frank Remembered (1995),"Young Poisoner's Handbook, The (1995)",If Lucy Fell (1996),"Steal Big, Steal Little (1995)","Boys of St. Vincent, The (1992)",Boomerang (1992),Chungking Express <Chung Hing sam lam> (1994),"Star Maker, The <Uomo delle stelle, L'> (1995)",Flirting With Disaster (1996),"NeverEnding Story III, The (1994)",Pie in the Sky (1996),Angela (1995),Frankie Starlight (1995),Jade (1995),Down Periscope (1996),Man of the Year (1995),Up Close and Personal (1996),"Birdcage, The (1996)","Brothers McMullen, The (1995)",Bad Boys (1995),"Amazing Panda Adventure, The (1995)","Basketball Diaries, The (1995)","Awfully Big Adventure, An (1995)",Amateur (1994),Apollo 13 (1995),Rob Roy (1995),"Addiction, The (1995)",Batman Forever (1995),Beauty of the Day <Belle de jour> (1967),Beyond Rangoon (1995),Blue in the Face (1995),Canadian Bacon (1995),Casper (1995),Clockers (1995),Congo (1995),Crimson Tide (1995),Crumb (1994),Desperado (1995),Devil in a Blue Dress (1995),Die Hard: With a Vengeance (1995),"Doom Generation, The (1995)",Feast of July (1995),First Knight (1995),Free Willy 2: The Adventure Home (1995),Hackers (1995),Jeffrey (1995),Johnny Mnemonic (1995),Judge Dredd (1995),Jury Duty (1995),Kids (1995),Living in Oblivion (1995),Lord of Illusions (1995),Love & Human Remains (1993),Mad Love (1995),Mall

In [108]:
X = util_matrix.copy(deep=True)
print(len(X.columns))
Y = X.dropna(axis=1, how='all')
print(len(Y.columns))

Y.head(10)

550
132


,userId,Toy Story (1995),Grumpier Old Men (1995),GoldenEye (1995),Sense and Sensibility (1995),Get Shorty (1995),Othello (1995),Dangerous Minds (1995),Twelve Monkeys <a.k.a. 12 Monkeys> (1995),Babe (1995),Dead Man Walking (1995),Clueless (1995),"Cry, the Beloved Country (1995)",To Die For (1995),Seven <a.k.a. Se7en> (1995),"Usual Suspects, The (1995)",Mighty Aphrodite (1995),"Indian in the Cupboard, The (1995)",Mr. Holland's Opus (1995),From Dusk Till Dawn (1996),Happy Gilmore (1996),Braveheart (1995),Taxi Driver (1976),Rumble in the Bronx <Hont faan kui> (1995),"NeverEnding Story III, The (1994)","Birdcage, The (1996)","Brothers McMullen, The (1995)",Apollo 13 (1995),Rob Roy (1995),"Addiction, The (1995)",Batman Forever (1995),Casper (1995),Crimson Tide (1995),Die Hard: With a Vengeance (1995),First Knight (1995),Free Willy 2: The Adventure Home (1995),Judge Dredd (1995),"Net, The (1995)",Nine Months (1995),Strange Days (1995),"Walk in the Clouds, A (1995)",Waterworld (1995),Circle of Friends (1995),Clerks (1994),Disclosure (1994),Dumb & Dumber <Dumb and Dumber> (1994),Ed Wood (1994),Heavenly Creatures (1994),Houseguest (1994),Interview with the Vampire: The Vampire Chronicles (1994),Star Wars: Episode IV - A New Hope (1977),Little Women (1994),Like Water for Chocolate <Como agua para chocolate> (1992),Legends of the Fall (1994),Major Payne (1995),"Madness of King George, The (1994)",Mary Shelley's Frankenstein <Frankenstein> (1994),Miracle on 34th Street (1994),Nell (1994),Only You (1994),Outbreak (1995),Léon: The Professional <a.k.a. The Professional Léon> (1994),Pulp Fiction (1994),Quiz Show (1994),"Secret of Roan Inish, The (1994)",Stargate (1994),"Santa Clause, The (1994)","Shawshank Redemption, The (1994)",Shallow Grave (1994),Star Trek: Generations (1994),Tommy Boy (1995),While You Were Sleeping (1995),Ace Ventura: Pet Detective (1994),"Adventures of Priscilla, Queen of the Desert, The (1994)",Clear and Present Danger (1994),"Client, The (1994)","Flintstones, The (1994)",Forrest Gump (1994),Four Weddings and a Funeral (1994),"Lion King, The (1994)","Mask, The (1994)",Naked Gun 33 1/3: The Final Insult (1994),"Paper, The (1994)",Reality Bites (1994),Speed (1994),True Lies (1994),Wolf (1994),Highlander III: The Sorcerer <a.k.a. Highlander: The Final Dimension> (1994),Addams Family Values (1993),Carlito's Way (1993),Cliffhanger (1993),Coneheads (1993),Dave (1993),Demolition Man (1993),"Firm, The (1993)","Fugitive, The (1993)",Hard Target (1993),"Englishman Who Went Up a Hill But Came Down a Mountain, The (1995)",In the Line of Fire (1993),Jurassic Park (1993),Last Action Hero (1993),Much Ado About Nothing (1993),Mrs. Doubtfire (1993),Philadelphia (1993),"Piano, The (1993)","Remains of the Day, The (1993)",Robin Hood: Men in Tights (1993),Rudy (1993),Schindler's List (1993),Shadowlands (1993),Sirens (1994),Sleepless in Seattle (1993),Blade Runner (1982),So I Married an Axe Murderer (1993),Threesome (1994),"Nightmare Before Christmas, The (1993)","Three Musketeers, The (1993)","Brady Bunch Movie, The (1995)",Home Alone (1990),Ghost (1990),Aladdin (1992),Terminator 2: Judgment Day (1991),Dances with Wolves (1990),Batman (1989),"Silence of the Lambs, The (1991)",Snow White and the Seven Dwarfs (1937),Beauty and the Beast (1991),Pinocchio (1940),Pretty Woman (1990),Fargo (1996),Heavy Metal (1981),"Aristocats, The (1970)"
0,2.0,NaN,NaN,1.5,2.5,NaN,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,1.5,1.5,0.5,NaN,0.5,NaN,NaN,1.5,NaN,NaN,NaN,NaN,0.5,2.5,NaN,NaN,1.5,NaN,0.5,0.5,0.5,NaN,NaN,0.5,0.5,NaN,NaN,0.5,2.5,-1.5,0.5,NaN,0.5,NaN,0.5,1.5,NaN,1.5,2.5,2.5,NaN,0.5,1.5,NaN,NaN,NaN,0.5,NaN,1.5,0.5,1.5,NaN,-0.5,NaN,-1.5,NaN,NaN,0.5,NaN,NaN,1.5,1.5,NaN,0.5,0.5,0.5,0.5,-0.5,0.5,0.5,0.5,NaN,0.5,-0.5,0.5,NaN,NaN,NaN,NaN,NaN,1.5,0.5,NaN,1.5,-0.5,1.5,0.5,0.5,1.5,1.5,1.5,1.5,NaN,NaN,1.5,NaN,1.5,0.5,NaN,NaN,0.5,2.5,0.5,2.5,0.5,0.5,0.5,2.5,2.5,2.5,0.5,NaN,NaN,NaN,NaN,NaN,NaN,0.5
1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN